# Event Study in Python
This program contains the codes that can be used in testing market reaction and doing sample partition. Event_CS.csv include the following variable:
-    **permno: security identifier**
-    **conm: company name**
-    **event_date: earnings announcement date**
-    **partition_var1: whether reported earnings beat analyst forecast consensus**
-    **partition_var2: whether firms report loss**

### Part A: Clean Data

**Loading pandas and dataset**

In [ ]:
import pandas as pd
import scipy.stats as st #Package for doing statistical test
import matplotlib.pyplot as plt #Package for drawing pictures
pd.options.mode.chained_assignment = None #Set off some pandas outputs

In [ ]:
#Import event data
event = pd.read_csv('Events_CS.csv',parse_dates = ['event_date'])
event

In [ ]:
#Duplicates check
event=event.drop_duplicates()
event

In [ ]:
#Import stock price data
stockprice = pd.read_csv('Crsp_2010_2020.csv',parse_dates = ['date','calendar_day'])

- permno: security identifier
- date: trading calendar day
- calendar_day: trading calendar day
- trading_day: trading day index
- abnormal_ret: raw return adjusted for delisting return and market return

**Preprocess dataset before merging**

In [ ]:
#Filter Event Firms' Stock Price
relatedsp=pd.merge(event, stockprice, how = 'left', on = 'permno')[['permno', 'date', 'abnormal_ret','calendar_day','trading_day']]

In [ ]:
#Duplicate Repetitive Merge
relatedsp=relatedsp.drop_duplicates()

In [ ]:
#Reset Index for Loop
event=event.reset_index(drop=True)
relatedsp=relatedsp.reset_index(drop=True)

**Merge each event to the closest trading day**

In [ ]:
#This code is to loop over each event and match each event with the cloest trading day within [0,+7].
event_trade_date = pd.DataFrame()
for r in range(0,len(event)):
    #Select event related price
    relatedsp_temp=relatedsp[relatedsp['permno']==event.loc[r,"permno"]] 
    #Assign event date to a seperate variable
    relatedsp_temp['event_date']=event.loc[r,'event_date'] 
    #Use the calendar_day minus event day
    relatedsp_temp['relative_day']=(relatedsp_temp['calendar_day']-relatedsp_temp['event_date']).dt.days 
    #Select match within [0,+7]
    relatedsp_temp=relatedsp_temp[(relatedsp_temp['relative_day']>=0)&(relatedsp_temp['relative_day']<=7)]
    #Sort and filter the trading day with minimum day difference
    relatedsp_temp=relatedsp_temp.sort_values(by = 'relative_day', ascending = True)
    temp=relatedsp_temp.head(1)[['permno', 'event_date', 'trading_day']]
    event_trade_date = event_trade_date.append(temp.head(1), ignore_index = True)
    
print("Number of Matched with CRSP:",len(event_trade_date))
event_trade_date.rename(columns={'trading_day':'event_trading_day'}, inplace=True)
print(event_trade_date)

**Collect event window [-5,10] trading data**

In [ ]:
#This code is to assign the 16 trading days around the event date, and transform each trading day to an event day.

#Merge the event with event_trading_day to price dataset 
event_price=pd.merge(event_trade_date, relatedsp, how = 'left', on = ['permno'])
#Calculate the trading day difference
event_price['event_day']=event_price['trading_day']-event_price['event_trading_day']
#Filter by graph window [-5,+10]
event_price=event_price[(event_price['event_day']<=10) & (event_price['event_day']>=-5)]
#Sort variable
event_price=event_price.sort_values(by = ['permno','event_date','event_day'], ascending = True)
event_price=event_price[['permno', 'event_date', 'calendar_day','trading_day','event_day','abnormal_ret']]
event_price.head(16)

**Delete events without sufficient trading data**

In [ ]:
event_price = event_price[event_price.groupby(['permno','event_date']).event_day.transform(len) ==16]

**Combine original data**

In [ ]:
temp1=pd.merge(event, event_price, how = 'left', on = ['permno','event_date'])

**Generate cumulative return**

In [ ]:
#For graphing, we need a longer window of [-5,10]

#Keep price data within window
graph_sample=temp1[(temp1['event_day']<=10) & (temp1['event_day']>=-5)] 
graph_sample
#Cumulatively sum the return
graph_sample['car'] = graph_sample.groupby(['permno','event_date'])['abnormal_ret'].apply(lambda x: x.cumsum()) 
graph_sample.shape

In [ ]:
#For testing, we need a shorter window of [-1,1]

#Keep price data within window
test_sample=temp1[(temp1['event_day']<=1) & (temp1['event_day']>=-1)]
#Cumulatively sum the return
test_sample['car'] = test_sample.groupby(['permno','event_date'])['abnormal_ret'].apply(lambda x: x.cumsum()) 
#Keep only one CAR observation for each event
test_sample=test_sample[test_sample['event_day']==1]
test_sample.shape

### Part B: Event Study Graph and Statistic Test

**Draw event study graph**

In [ ]:
mean_ar=graph_sample.groupby('event_day')['car'].mean()
plt.plot(mean_ar)

**Statistical test**

In [ ]:
#Print average car
print(test_sample['car'].mean())
#Do the test and print t and p values
t, p = st.ttest_1samp(test_sample['car'], 0)
print("ttest_1samp_stats: t = %.3f  p = %.3f" % (t, p))

### Part C: Sample Partition

**Partition graph**

In [ ]:
#Split the sample based on partition variable
group1=graph_sample[graph_sample['partition_var1']==0]
group2=graph_sample[graph_sample['partition_var1']==1]

In [ ]:
#Draw picture for group 1
mean_ar=group1.groupby('event_day')['car'].mean()
plt.plot(mean_ar)

In [ ]:
#Draw picture for group 2
mean_ar=group2.groupby('event_day')['car'].mean()
plt.plot(mean_ar)

**Test market reaction for each group**

In [ ]:
#Split the sample based on partition variable
group1=test_sample[test_sample['partition_var1']==0]
group2=test_sample[test_sample['partition_var1']==1]

In [ ]:
#Average Car for group 1
print("Average Market Reaction: car = %.3f" % group1['car'].mean())

In [ ]:
#Average Car for group 2
print("Average Market Reaction: car = %.3f" % group2['car'].mean())

In [ ]:
#Test the difference across group

#Two sample test
t,p=st.ttest_ind(group1['car'], group2['car'])
print("Comparing Two Groups' Car: t = %.3f  p = %.3f" % (t, p))
